In [1]:
require 'stopwords'
require 'rblearn'
require 'stemmify'
require 'numo/narray'
require 'rblearn'

false

# 70. データの入手・整形

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [2]:
def knock70
  pos = File.open("../data/rt-polaritydata/rt-polarity.pos").read
  neg = File.open("../data/rt-polaritydata/rt-polarity.pos").read

  pos_tagged = pos.split("\n").map {|line| "+1 #{line}"}
  neg_tagged = neg.split("\n").map {|line| "-1 #{line}"}
  sen = (pos_tagged + neg_tagged).shuffle
  counter = {"+1" => 0, "-1" => 0}

  out = File.open("../data/sentiment.txt", "w")
  sen.each do |line|
    out.puts line
  end
  out.close

  open('../data/sentiment.txt').each_line do |line|
    counter[line[0..1]] += 1
  end

  return counter
end

:knock70

In [3]:
knock70

{"+1"=>5330, "-1"=>5330}

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [4]:
# in order to check if the word is stop word with O(1)
Stop_words = Hash.new {|hash, key| hash[key] = false}
(Stopwords::STOP_WORDS + ["'", ",", '.', ":", ";"]).each {|stop_word| Stop_words[stop_word] = true}

def is_stop?(word)
  Stop_words[word]
end

:is_stop?

# 72. 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [5]:
def knock72
  labels = []
  features = []

  File.foreach('../data/sentiment.txt') do |line|
    line.chomp!
    label = line[0..1]
    feature = line[3..-1]
    labels << (label == '+1' ? 1 : 0)
    features << feature
  end

  cv = Rblearn::CountVectorizer.new(lambda{|feature| feature.split.map(&:stem).map(&:downcase)}, 1, 0.7)
  cv.fit_transform(features)

  y = Numo::Int8.zeros([labels.size, 1])
  labels.each_with_index do |label, i|
    y[i, 0] = label
  end

  return [cv, y]
end

:knock72

In [6]:
cv, y = knock72

1

1

# 73. 学習

72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [7]:
class LogisticRegressionModel
  attr_accessor :w

  def initialize(eta, threshold)
    @eta = eta
    @threshold = threshold
  end
  
  
  def h(x)
    z = x.dot(@w)
    return 1 / (1 + Numo::NMath.exp(-z))
  end


  def fit(x, y)
    # @w = Numo::Float64.zeros([x.shape[1], 1])
    @w = Numo::Float64.new([x.shape[1], 1]).rand
    60.times do |t|

      # p y-h(x)
      # NOTE: there is something bug
      @w = @w - @eta * x.transpose.dup.dot(h(x) - y) # x.T.dot(h(x) - y): gradient
      puts "#iter: #{t}, accuracy: #{score(x, y)} loss: #{loss(x, y)}" if t % 5 == 0
    end
  end


  def predict(x)
    probabilities = h(x)
    return probabilities > @threshold
  end


  def predict_prob(x)
    return h(x)
  end


  def score(x, y)
    predicted_class = predict(x)
    num_data = predicted_class.shape[0]

    py = predicted_class.to_a
    ry = y.to_a

    num_correct = 0
    num_data.times do |t|
      num_correct += 1 if (py[t] == ry[t])
    end

    return num_correct.to_f / num_data
  end


  def loss(x, y)
    a = - Numo::NMath.log(h(x)).transpose.dot(y)[0, 0]
    b = - Numo::NMath.log(1-h(x)).transpose.dot(1-y)
    (a + b)[0, 0]
  end
end

:loss

In [8]:
x = cv.doc_matrix
x_train, y_train, x_test, y_test = Rblearn::CrossValidation.train_test_split(x, y, 0.3).map(&:dup)
model = LogisticRegressionModel.new(0.01, 0.5)
model.fit(x_train, y_train)

puts "test accuracy: #{model.score(x_test, y_test)}"

#iter: 0, accuracy: 0.49705172875904585 loss: 10786.126143069476
#iter: 5, accuracy: 0.519163763066202 loss: 6281.452210917492
#iter: 10, accuracy: 0.536853390511927 loss: 5668.623745819228
#iter: 15, accuracy: 0.5501206110962209 loss: 5356.689439940982
#iter: 20, accuracy: 0.5620477083891718 loss: 5164.521007243623
#iter: 25, accuracy: 0.5722326454033771 loss: 5032.939191396279
#iter: 30, accuracy: 0.5789332618600911 loss: 4936.112883462274
#iter: 35, accuracy: 0.5871080139372822 loss: 4860.962584956831
#iter: 40, accuracy: 0.5940766550522648 loss: 4800.293747277765
#iter: 45, accuracy: 0.5998391852050389 loss: 4749.8715454125595
#iter: 50, accuracy: 0.6050656660412758 loss: 4707.019934522199
#iter: 55, accuracy: 0.6093540605735728 loss: 4669.937993763386
test accuracy: 0.23827392120075047


# 74. 予測

73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

In [9]:
model.predict(x_test).each_with_index do |arr, index|
  puts arr
  break if index == 30
end

0
1
0
1
1
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
0
1
0
1
0
1
1
0
1
1


# 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

In [10]:
feature_names = cv.get_feature_names
weights = model.w.to_a.map{|arr| arr[0]}
name_weights = {}

feature_names.size.times do |t|
  name_weights[feature_names[t]] = weights[t]
end

puts "high"
name_weights.
  sort{|(_, v1), (_, v2)| v2 <=> v1}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

puts "low"
name_weights.
  sort{|(_, v1), (_, v2)| v1 <=> v2}.
  first(10).
  map{|arr| arr.join("\t")}.
  each do |record|
    puts record
  end

1

high
hilar	1.4466232193946458
intellect	1.3428097121689064
jame	1.3211350038163003
forest	1.3000183981011286
trashi	1.2731607045727318
categori	1.2699986591495904
guilt	1.265813439281444
entri	1.2658079361121046
resist	1.2541875840938255
unhurri	1.2403106858150077
low
neil	-1.2406206446496397
o	-1.1550142050451178
georg	-1.148662530126893
hous	-1.1470415360623498
liber	-1.0491716309697714
greatest	-1.0379896738077525
left	-1.0378007072739752
j	-0.9859741311207221
everydai	-0.9603967144056813
safe	-0.9561737051588816


1

# 76. ラベル付け

学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

In [11]:
a = y_test.to_a.map{|arr| arr[0]}
b = model.predict(x_test).to_a.map{|arr| arr[0]}
c = model.predict_prob(x_test).to_a.map{|arr| arr[0]}

output = open('../data/knock76.tsv', 'w')

a.size.times do |t|
  puts "#{a[t]}\t#{b[t]}\t#{c[t]}" if t < 30
  output.puts "#{a[t]}\t#{b[t]}\t#{c[t]}"
end

output.close

1	0	0.3790063678703021
0	1	0.7169789416107691
0	0	0.4980758602896097
0	1	0.7768226254367465
0	1	0.5438573472933226
1	0	0.05073617354228076
0	0	0.3858109211434544
1	0	0.3467848833474927
1	0	0.36112053854763615
1	0	0.37108134886668903
1	1	0.778321592637828
1	0	0.12048240992930448
0	1	0.5429813677467128
0	1	0.8154461191796761
0	1	0.9031194760921107
0	1	0.8930828783199266
0	1	0.7431465813170596
0	1	0.7943655129917341
0	1	0.811473221202199
1	1	0.7380209305903574
1	1	0.7822330881510614
0	0	0.45851082057535825
0	1	0.7375629473664235
1	0	0.2436159602191
0	1	0.9108123427268369
0	0	0.4216989879629056
0	1	0.7605618773106952
0	1	0.5192097437511775
0	0	0.21156060898586398
0	1	0.6111221582961177


# 77. 正解率の計測

76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

In [12]:
num_all = 0
num1 = 0.0
num2 = 0.0
num3 = 0.0

# a is the actual label
# b is the predicted label
File.foreach('../data/knock76.tsv') do |line|
  num_all += 1

  a, b, _ = line.split.map(&:to_f)
  
  if b == 1
    if a == 1
      num1 += 1
    elsif a == 0
      num2 += 1
    end
  elsif b == 0 && a == 1
    num3 += 1
  end
end

precision = num1 / (num1 + num2)
recall = num3 / (num1 + num3)

puts "precision: #{precision}"
puts "recall: #{recall}"
puts "F-1 score: #{2*precision*recall / (precision + recall)}"

precision: 0.25
recall: 0.7472187886279357
F-1 score: 0.3746513789897738
